In [1]:
from pymysql import connect
from pandas import DataFrame
from numpy import zeros, int64, int32, float64, float32, multiply, dot, identity, sum
from itertools import permutations

In [2]:
db_name = input()

mondial


In [3]:
db = db_name
connection = connect(host="localhost", user="root", password="", db=db)
cursor = connection.cursor()
db_setup = db_name + "_setup"
connection_setup = connect(host="localhost", user="root", password="", db=db_setup)
cursor_setup = connection_setup.cursor()
db_bn = db_name + "_bn"
connection_bn = connect(host="localhost", user="root", password="", db=db_bn)
cursor_bn = connection_bn.cursor()

In [4]:
keys = {}

In [5]:
cursor_setup.execute("SELECT TABLE_NAME FROM entitytables");
entity_tables = cursor_setup.fetchall()
entities = {}
for i in entity_tables:
    cursor.execute("SELECT * FROM " + i[0])
    rows = cursor.fetchall()
    cursor.execute("SHOW COLUMNS FROM " + db + "." + i[0])
    columns = cursor.fetchall()
    entities[i[0]] = DataFrame(rows, columns=[columns[j][0] for j in range(len(columns))])
    cursor_setup.execute("SELECT COLUMN_NAME FROM entitytables WHERE TABLE_NAME = " + "'" + i[0] + "'")
    key = cursor_setup.fetchall()
    keys[i[0]] = key[0][0]

In [6]:
cursor_setup.execute("SELECT TABLE_NAME FROM relationtables")
relation_tables = cursor_setup.fetchall()
relations = {}
for i in relation_tables:
    cursor.execute("SELECT * FROM " + i[0])
    rows = cursor.fetchall()
    cursor.execute("SHOW COLUMNS FROM " + db + "." + i[0])
    columns = cursor.fetchall()
    relations[i[0]] = DataFrame(rows, columns=[columns[j][0] for j in range(len(columns))])
    cursor_setup.execute("SELECT COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = " + "'" + i[0] + "'")
    key = cursor_setup.fetchall()
    keys[i[0]] = key[0][0], key[1][0]

In [7]:
relation_names = tuple(i[0] for i in relation_tables)

In [8]:
indices = {}
for i in entity_tables:
    cursor_setup.execute("SELECT COLUMN_NAME FROM entitytables WHERE TABLE_NAME = '" + i[0] + "'")
    key = cursor_setup.fetchall()[0][0]
    indices[key] = {}
    for index, row in entities[i[0]].iterrows():
        indices[key][row[key]] = index

In [9]:
matrices = {}
for i in relation_tables:
    cursor_setup.execute("SELECT REFERENCED_TABLE_NAME FROM foreignkeycolumns WHERE TABLE_NAME = " + "'" + i[0] + "'")
    reference = cursor_setup.fetchall()
    matrices[i[0]] = zeros((len(entities[reference[0][0]].index), len(entities[reference[1][0]].index)))

In [10]:
for i in relation_tables:
    cursor_setup.execute("SELECT COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = '" + i[0] + "'")
    key = cursor_setup.fetchall()
    cursor_setup.execute("SELECT COLUMN_NAME, REFERENCED_COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = '" + i[0] + "'")
    reference = cursor_setup.fetchall()
    for index, row in relations[i[0]].iterrows():
        matrices[i[0]][indices[reference[0][1]][row[key[0][0]]]][indices[reference[1][1]][row[key[1][0]]]] = 1

In [11]:
cursor_setup.execute("SELECT COLUMN_NAME, TABLE_NAME FROM attributecolumns")
attribute_columns = cursor_setup.fetchall()
attributes = {}
for i in attribute_columns:
    attributes[i[0]] = i[1]

In [12]:
cursor_bn.execute("SELECT DISTINCT child FROM final_path_bayesnets_view")
childs = cursor_bn.fetchall()
rules = []
multiples = []
states = []
functors = {}
variables = {}
nodes = {}
masks = {}
base_indices = []
mask_indices = []
sort_indices = []
stack_indices = []
values = []
for i in range(len(childs)):
    rule = [childs[i][0]]
    cursor_bn.execute("SELECT parent FROM final_path_bayesnets_view WHERE child = " + "'" + childs[i][0] + "'")
    parents = cursor_bn.fetchall()
    for j in parents:
        if j[0] != '':
            rule += [j[0]]
    rules.append(rule)
    if len(rule) == 1:
        multiples.append(0)
    else:
        multiples.append(1)
    relation_check = 0
    for j in rule:
        if j.find(',') != -1:
            relation_check = 1
    functor = {}
    variable = {}
    node = {}
    state = []
    mask = {}
    unmasked_variables = []
    for j in range(len(rule)):
        fun = rule[j].split('(')[0]
        functor[j] = fun
        if rule[j].find(',') == -1:
            var = rule[j].split('(')[1][:-1]
            variable[j] = var
            node[j] = var[:-1]
            if relation_check == 0:
                unmasked_variables.append(var)
                state.append(0)
            else:
                mas = []
                for k in rule:
                    func = k.split('(')[0]
                    if func not in relation_names:
                            func = attributes[func]
                    if k.find(',') != -1 and k.find(var) != -1:
                        unmasked_variables.append(k.split('(')[1][:-1])
                        mas.append([func, k.split('(')[1].split(',')[0], k.split('(')[1].split(',')[1][:-1]]) 
                mask[j] = mas
                state.append(1)
        else:
            unmasked_variables.append(rule[j].split('(')[1][:-1])
            if fun in relation_names:
                state.append(2)
            else:
                state.append(3)     
    functors[i] = functor
    variables[i] = variable
    nodes[i] = node
    states.append(state)
    masks[i] = mask
    masked_variables = [unmasked_variables[0]]
    base_indice = [0]
    mask_indice = []
    for j in range(1, len(unmasked_variables)):
        mask_check = 0
        for k in range(len(masked_variables)):
            if unmasked_variables[j] == unmasked_variables[k]:
                mask_indice.append([k, j])
                mask_check = 1
        if mask_check == 0:
            base_indice.append(j)
            masked_variables.append(unmasked_variables[j])
    sort_indice = []
    sorted_variables = []
    if relation_check == 0:
        sort_indice.append([False, 0])
        sorted_variables.append(masked_variables[0])
    else:
        indices_permutations = list(permutations(range(len(masked_variables))))
        variables_permutations = list(permutations(masked_variables))
        for j in range(len(variables_permutations)):
            indices_chain = []
            variables_chain = []
            first = variables_permutations[j][0].split(',')[0]
            second = variables_permutations[j][0].split(',')[1]
            indices_chain.append([False, indices_permutations[j][0]])
            variables_chain.append(variables_permutations[j][0])
            untransposed_check = 1
            transposed_check = 1
            if len(variables_permutations[j]) > 1:
                for k in range(1, len(variables_permutations[j])):
                    next_first = variables_permutations[j][k].split(',')[0]
                    next_second = variables_permutations[j][k].split(',')[1]
                    if second == next_first:
                        second = next_second
                        indices_chain.append([False, indices_permutations[j][k]])
                        variables_chain.append(next_first + ',' + next_second)
                    elif second == next_second:
                        second = next_first
                        indices_chain.append([True, indices_permutations[j][k]])
                        variables_chain.append(next_second + ',' + next_first)    
                    else:
                        untransposed_check = 0
                        break
                if untransposed_check != 1:
                    indices_chain[0] = [True, indices_permutations[j][0]]
                    variables_chain[0] = second + ',' + first
                    temp = first
                    first = second
                    second = temp
                    for k in range(1, len(variables_permutations[j])):
                        next_first = variables_permutations[j][k].split(',')[0]
                        next_second = variables_permutations[j][k].split(',')[1]
                        if second == next_first:
                            second = next_second
                            indices_chain.append([False, indices_permutations[j][k]])
                            variables_chain.append(next_first + ',' + next_second)
                        elif second == next_second:
                            second = next_first
                            indices_chain.append([True, indices_permutations[j][k]])
                            variables_chain.append(next_second + ',' + next_first)    
                        else:
                            transposed_check = 0
                            break
            if untransposed_check == 1 or transposed_check == 1 or len(variables_permutations[j]) == 1:
                sort_indice = indices_chain
                sorted_variables = variables_chain
                break    
    stack_indice = []
    for j in range(1, len(sorted_variables)):
        second = sorted_variables[j].split(',')[1]
        for k in range(j - 1, -1, -1):
            previous_first = sorted_variables[k].split(',')[0]
            if previous_first == second:
                stack_indice.append([k, j])   
    base_indices.append(base_indice)
    mask_indices.append(mask_indice)
    sort_indices.append(sort_indice)
    stack_indices.append(stack_indice)
    cursor_bn.execute("SELECT * FROM `" + childs[i][0] + "_cp`")
    value = cursor_bn.fetchall()
    values.append(value)

In [13]:
for i in range(len(rules)):
    print(rules[i])
    for j in values[i]:
        unmasked_matrices = []
        for k in range(len(rules[i])):
            match states[i][k]:
                case 0:
                    matrix = zeros((len(entities[nodes[i][k]].index), 1))
                    for l in range(len(entities[nodes[i][k]][functors[i][k]])):
                        value = entities[nodes[i][k]][functors[i][k]][l]
                        if type(j[k+multiples[i]]) == str:
                            if type(value) == int64 or type(value) == int32:
                                value = str(value)
                            elif type(value) == float64 or type(value) == float32:
                                value = str(int(value))
                        if value == j[k+multiples[i]]:
                            matrix[indices[keys[nodes[i][k]]][entities[nodes[i][k]][keys[nodes[i][k]]][l]]][0] = 1
                    unmasked_matrices.append(matrix)
                case 1:
                    for l in masks[i][k]:
                        matrix = zeros(matrices[l[0]].shape)
                        for m in range(len(entities[nodes[i][k]][functors[i][k]])):
                            if entities[nodes[i][k]][functors[i][k]][m] == j[k+multiples[i]]:
                                if variables[i][k] == l[1]:
                                    matrix[indices[keys[nodes[i][k]]][entities[nodes[i][k]][keys[nodes[i][k]]][m]],:] = 1
                                elif variables[i][k] == l[2]:
                                    matrix[:,indices[keys[nodes[i][k]]][entities[nodes[i][k]][keys[nodes[i][k]]][m]]] = 1
                        unmasked_matrices.append(matrix)
                case 2:
                    if j[k+multiples[i]] == 'F':
                        unmasked_matrices.append(1 - matrices[functors[i][k]])
                    else:
                        unmasked_matrices.append(matrices[functors[i][k]])
                case 3:
                    if j[k+multiples[i]] == 'N/A':
                        unmasked_matrices.append(1 - matrices[attributes[functors[i][k]]])
                    else:
                        matrix = zeros(matrices[attributes[functors[i][k]]].shape)
                        for l in range(len(relations[attributes[functors[i][k]]][functors[i][k]])):
                            if relations[attributes[functors[i][k]]][functors[i][k]][l] == j[k+multiples[i]]:
                                matrix[indices[keys[attributes[functors[i][k]]][0]][relations[attributes[functors[i][k]]][keys[attributes[functors[i][k]]][0]][l]]][indices[keys[attributes[functors[i][k]]][1]][relations[attributes[functors[i][k]]][keys[attributes[functors[i][k]]][1]][l]]] = 1 
                        unmasked_matrices.append(matrix)
        masked_matrices = []
        for k in base_indices[i]:
            masked_matrices.append(unmasked_matrices[k])
        for k in mask_indices[i]:
            masked_matrices[k[0]] = multiply(masked_matrices[k[0]], unmasked_matrices[k[1]])
        sorted_matrices = []
        for k in sort_indices[i]:
            if k[0]:
                sorted_matrices.append(masked_matrices[k[1]].T)
            else:
                sorted_matrices.append(masked_matrices[k[1]])
        stacked_matrices = sorted_matrices.copy()   
        pop_counter = 0
        for k in stack_indices[i]:
            for l in range(k[1] - k[0] - pop_counter):
                stacked_matrices[k[0]] = dot(stacked_matrices[k[0]], stacked_matrices[k[0] + 1])
                stacked_matrices.pop(k[0] + 1)
                pop_counter += 1
            stacked_matrices[k[0]] = multiply(stacked_matrices[k[0]], identity(len(stacked_matrices[k[0]])))
        result = stacked_matrices[0]
        for k in range(1, len(stacked_matrices)):
            result = dot(result, stacked_matrices[k])
        print(sum(result))
    print("---------------------------------------------------------------------------------------------------------------")

['agricul(eco0)', 'ecor(country0,eco0)', 'ecor(country1,eco0)', 'service(eco0)']
136162.0
169280.0
203505.0
203505.0
238099.0
368.0
920.0
920.0
920.0
736.0
368.0
920.0
920.0
920.0
736.0
2.0
5.0
5.0
5.0
4.0
101568.0
33856.0
169280.0
236992.0
372416.0
552.0
184.0
920.0
1288.0
2024.0
552.0
184.0
920.0
1288.0
2024.0
3.0
1.0
5.0
7.0
11.0
101937.0
101568.0
270848.0
304704.0
169280.0
368.0
552.0
1472.0
1656.0
920.0
368.0
552.0
1472.0
1656.0
920.0
2.0
3.0
8.0
9.0
5.0
169649.0
305442.0
236992.0
135424.0
101568.0
736.0
1288.0
1288.0
736.0
552.0
736.0
1288.0
1288.0
736.0
552.0
4.0
7.0
7.0
4.0
3.0
406641.0
338929.0
33856.0
33856.0
2024.0
1656.0
184.0
184.0
2024.0
1656.0
184.0
184.0
11.0
9.0
1.0
1.0
---------------------------------------------------------------------------------------------------------------
['border(country0,country1)', 'continent(country1)', 'percentage(country0)']
1656.0
1656.0
1691.0
2600.0
773.0
1476.0
1462.0
1471.0
2250.0
673.0
1393.0
1401.0
1378.0
2118.0
641.0
740.0
740.0
7